# Get the Exact Answer
Start off by computing the exact Hessian to use a reference point. 
First relax the structure then compute the Hessians using [ase's Vibrations module](https://databases.fysik.dtu.dk/ase/ase/vibrations/modes.html), which will compute them numerically using central derivatives

In [1]:
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import VibrationsData, Vibrations
from ase.calculators.mopac import MOPAC
from ase.calculators.psi4 import Psi4
from ase.optimize import QuasiNewton
from ase import Atoms, units
from ase.io import write
from time import perf_counter
from platform import node
from pathlib import Path
import numpy as np
import shutil
import json
import os

Configuration

In [2]:
molecule_name = 'water'
run_analytic = True
deltas = [0.0005, 0.002, 0.005, 0.008, 0.01, 0.012, 0.02, 0.05]
fmax = .01
#method = 'ccsd(t)'
#basis = 'cc-pvdz'
levels = [
    ['hf', 'def2-svpd'], 
    ['hf', '6-31g*'],
    ['b3lyp','6-31g*'],
    ['b3lyp','cc-pvtz'],
    ['wb97x-d','cc-pvtz'], 
    ['m062x', 'cc-pvtz'],
    ['ccsd(t)','cc-pvdz']]
method = levels[6][0]
basis = levels[6][1]
mopac_methods=['pm6', 'pm7']
#molecule_name = 'water'
#method = 'pm7'
#mopac_methods = ['pm7']  # Use MOPAC for these methods
#basis = None  # Set to None for MOPAC methods
threads = min(os.cpu_count(), 12)
#assert (method in mopac_methods) == (basis is None), 'Use a basis of None for MOPAC computations'

Derived

In [3]:
run_name = f'{molecule_name}_{method}_{basis}'

## Load in Target Molecule
We have it in a JSON file from PubChem

In [4]:
def load_molecule(name: str) -> Atoms:
    """Load a molecule from a PubChem JSON file
    
    Args:
        name: Name of the molecule
    Returns:
        ASE Atoms object
    """
    
    # Get the compound data
    with open(f'../data/structures/{name}.json') as fp:
        data = json.load(fp)
    data = data['PC_Compounds'][0]
        
    # Extract data from the JSON
    atomic_numbers = data['atoms']['element']
    positions = np.zeros((len(atomic_numbers), 3))
    conf_data = data['coords'][0]['conformers'][0]
    for i, c in enumerate('xyz'):
        if c in conf_data:
            positions[:, i] = conf_data[c]
        
    # Build the object    
    return Atoms(numbers=atomic_numbers, positions=positions)

In [5]:
atoms = load_molecule(molecule_name)

## Perform the Geometry Optimization
Build the ASE calculator then run QuasiNewton to a high tolerance

In [6]:
if method not in mopac_methods:
    calc = Psi4(method=method, basis=basis, num_threads=threads, memory='4096MB')
else:
    calc = MOPAC(method=method, command='mopac PREFIX.mop > /dev/null')

KeyboardInterrupt: 

In [ ]:
%%time
atoms.calc = calc
dyn = QuasiNewton(atoms)
dyn.run(fmax=fmax)

Save the output file

In [ ]:
out_dir = Path('../data') / 'exact'
out_dir.mkdir(exist_ok=True)

In [ ]:
write(out_dir / f'{run_name}.xyz', atoms)

## Compute the Hessian using ASE
ASE has a built-in method which uses finite displacements

In [ ]:
for delta in deltas:
    if Path('vib').is_dir():
        shutil.rmtree('vib')
    finite_diff_time = perf_counter()
    vib = Vibrations(atoms, delta=delta)
    vib.run()
    finite_diff_time = perf_counter() - finite_diff_time
    vib_data = vib.get_vibrations()
    with (out_dir / f'{run_name}-{delta}-ase.json').open('w') as fp:
        vib_data.write(fp)
    vib_data.get_zero_point_energy()

Save the vibration data

Print the ZPE for reference

## Repeat with Psi4's analytic derivatives
See if we get the same answer faster

In [ ]:
if isinstance(calc, Psi4) and run_analytic:
    for level in levels:
        print(out_dir)
        method, basis = level
        run_name = f'{molecule_name}_{method}_{basis}'
        ## Compute
        # # Get the compound data
        #with open(f'../data/structures/{molecule_name}.json') as fp:
        #    data = json.load(fp)
        #data = data['PC_Compounds'][0]
        #
        ## Extract data from the JSON
        #atomic_numbers = data['atoms']['element']
        #positions = np.zeros((len(atomic_numbers), 3))
        #conf_data = data['coords'][0]['conformers'][0]
        #for i, c in enumerate('xyz'):
        #    if c in conf_data:
        #        positions[:, i] = conf_data[c]
           
        # Build the object    
        calc = Psi4(method=method, basis=basis, num_threads=threads, memory='4096MB')
        atoms.calc = calc
        dyn = QuasiNewton(atoms)
        dyn.run(fmax=0.01)
        atoms = Atoms(numbers=atomic_numbers, positions=positions)
        calc.set_psi4(atoms)
        hess = calc.psi4.hessian(f'{method}/{basis}')
        # Convert to ASE format
        analytic_hess = hess.to_array() * units.Hartree / units.Bohr / units.Bohr
        vib_data = VibrationsData.from_2d(atoms, analytic_hess)
        with (out_dir / f'{run_name}-analytic.json').open('w') as fp:
            vib_data.write(fp)
        # with (out_dir / f'{run_name}-times.json').open('w') as fp:
        #    json.dump({
        #        'hostname': node(),
        #        'finite-diff': finite_diff_time,
        #        'analytic': analytic_time,
        #    }, fp)


In [ ]:
with (out_dir / f'{run_name}-times.json').open('w') as fp:
    json.dump({
        'hostname': node(),
        'finite-diff': finite_diff_time,
        'analytic': analytic_time,
    }, fp)